In [ ]:
import torch
import scipy.io
import sklearn
import math
import numpy as np
# # FTLOptimizer 
# class FTLOptimizer(torch.optim.Optimizer): 
      
#     # Init Method: 
#     def __init__(self, params, lr=1e-3): 
#         super(FTLOptimizer, self).__init__(params, defaults={'lr': lr}) 
#         self.state = dict() 
#         for group in self.param_groups: 
#             for p in group['params']: 
#                 self.state[p] = dict(mom=torch.zeros_like(p.data)) 
      
#     # Step Method 
#     def step(self): 
#         for group in self.param_groups: 
#             for p in group['params']: 
#                 if p not in self.state: 
#                     self.state[p] = dict(mom=torch.zeros_like(p.data)) 
#                 mom = self.state[p]['mom'] 
#                 mom = self.momentum * mom - group['lr'] * p.grad.data 
#                 p.data += mom

In [ ]:
mat = scipy.io.loadmat("spam_data.mat")
spam_inst = mat["spam_inst"]
spam_label = mat["spam_label"][:, 0]



T = len(spam_label)
print(T)
B = 2
L = 1

eta = 2 / (math.sqrt(2 * T))
known_spam_inst = []
known_spam_label = []
class_n=0
class_p=0
prediction=[]
# FTRL
for i in range(len(spam_label)):
    if (class_n>0)+(class_p>0)!=2:
        if spam_label[i]==-1:
            class_n+=1
        else:
            class_p+=1
        if class_n>class_p:
            prediction.append(-1)
        else:
            prediction.append(1)
    else:
        model = sklearn.svm.LinearSVC(loss="hinge", C=eta, dual='auto')
        model.fit(np.array(known_spam_inst), np.array(known_spam_label))
        p=model.predict(spam_inst[i].reshape(1, -1))
        prediction.append(p)
    
    known_spam_inst.append(spam_inst[i])
    known_spam_label.append(spam_label[i])

model = sklearn.svm.LinearSVC(loss="hinge", C=eta*100, dual='auto')
model.fit(np.array(spam_inst), np.array(spam_label))
prediction2=model.predict(spam_inst)
print(sklearn.metrics.accuracy_score(spam_label,prediction2))

4601
0.7200608563355793


c:\Users\lenovo\anaconda3\envs\dl\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
